In [1]:
!pip install umap-learn
!git clone https://github.com/google-research/soft-dtw-divergences.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82816 sha256=d5ab30c87b1a985623e791c1e4de67f9278a5c9045e1481733ba7ad548bb0212
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55622 sha256=fc16dee2758eaf0f3c8822571113bacd85a51867c0805cc60183d9dec4371385
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent
Cloning into 'soft-dtw-divergences'...
remote: Enumerating objects: 26, done.
remote

In [2]:
import sys
sys.path.append('/content/soft-dtw-divergences')


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import os
import pickle
import numpy as np
from sdtw_div.numba_ops_cuda import sdtw_div, sdtw_div_value_and_grad
import functools
import numba
from numba import cuda


In [ ]:
%%time
reducer = cuml.UMAP(
    n_neighbors=15,
    n_components=2,
    n_epochs=500,
    min_dist=0.1
)
embedding = reducer.fit_transform(data)

In [ ]:
# import packages
import umap
import numpy as np

import pandas as pd
from numba import njit, prange
import matplotlib.pyplot as plt
import os
import pickle
from sklearn.model_selection import ParameterGrid
import csv
import time
from sdtw_div.numba_ops import sdtw_div, sdtw_div_value_and_grad
import functools


In [12]:
import numba as nb
import sys


def calculate_divergence_matrix(X, gamma):
    m = X.shape[0]
    divergence_matrix = np.zeros((m, m))

    total_calculations = m * m
    percent_interval = total_calculations // 100  # Print progress every 1% of calculations

    for i in range(m):
        for j in range(m):
            divergence_matrix[i, j] = sdtw_div(X[i].reshape(-1, 1), X[j].reshape(-1, 1), gamma)

            if (i * m + j + 1) % percent_interval == 0:
                progress_percent = (i * m + j + 1) / total_calculations * 100
                sys.stdout.write("Progress: {:.2f}%".format(progress_percent))
                sys.stdout.flush()

    sys.stdout.write("\n")

    return divergence_matrix

# @cuda.jit
# def calculate_divergence_matrix(X, gamma, divergence_matrix):
#     i, j = cuda.grid(2)
#     if i < X.shape[0] and j < X.shape[0]:
#         divergence_matrix[i, j] = sdtw_div(X[i].reshape(-1, 1), X[j].reshape(-1, 1), gamma)

# def calculate_divergence_matrix_gpu(X, gamma):
#     m = X.shape[0]
#     divergence_matrix = np.zeros((m, m))

#     total_calculations = m * m
#     percent_interval = total_calculations // 100  # Print progress every 1% of calculations

#     threadsperblock = (16, 16)
#     blockspergrid_x = (m + threadsperblock[0] - 1) // threadsperblock[0]
#     blockspergrid_y = (m + threadsperblock[1] - 1) // threadsperblock[1]
#     blockspergrid = (blockspergrid_x, blockspergrid_y)

#     d_X = cuda.to_device(X)
#     d_divergence_matrix = cuda.to_device(divergence_matrix)

#     calculate_divergence_matrix[blockspergrid, threadsperblock](d_X, gamma, d_divergence_matrix)

#     d_divergence_matrix.copy_to_host(divergence_matrix)

#     return divergence_matrix

In [10]:
# Define the directory where the pickle files are located
base_dir = "/content/drive/My Drive/00_project/05_fly-arena/10_Locomotion/"
pickle_directory = base_dir + 'output_windows/50/'


In [ ]:
for filename in os.listdir(pickle_directory):
        print(f"Processing file: {filename}")
        if filename.endswith('.pkl'):
            # Extract the window size from the filename
            window_size = int(filename.split('_')[3])

            # Read the pickle file
            with open(os.path.join(pickle_directory, filename), 'rb') as f:
                data = pickle.load(f)

            # Select columns based on window_size
            columns = data.columns[:window_size * 4]

            print(f"DataFrame selected... column size: {window_size * 4}")
            data = data[columns].values
            gamma = 0.1

            # Call the kernel with the launch configuration
            calculate_divergence_matrix(data, gamma)

            # Calculate the distance matrix using parallel processing
            distance_matrix = calculate_divergence_matrix(data, gamma)

            print(distance_matrix)

Processing file: bootstrap_50_win_17_fly_10_exp_10_strain_p65GAL4TNT_20230606_184810.pkl
DataFrame selected... column size: 68
Progress: 1.00%Progress: 2.00%Progress: 3.00%Progress: 4.00%

In [ ]:


# Generate all possible combinations of hyperparameters
param_combinations = ParameterGrid(param_grid)

best_variance_mean = float('inf')
best_params = None
top_5_scores = []

# Iterate over each combination and calculate the scores
for params in param_combinations:
    print(f"Processing hyperparameters: {params}")
    for filename in os.listdir(pickle_directory):
        print(f"Processing file: {filename}")
        if filename.endswith('.csv'):
            # Extract the window size from the filename
            window_size = int(filename.split('_')[3])

            # Read the pickle file
            with open(os.path.join(pickle_directory, filename), 'rb') as f:
                data = pd.read_csv(f)

            # Select columns based on window_size
            columns = data.columns[:window_size * 4]

            print(f"DataFrame selected... column size: {window_size * 4}")
#             data = data[columns].values[:2500, :]
            data = data[columns].values

#             # Specify the number of rows to sample
#             num_samples = 4000

#             # Randomly select rows from the data
#             random_indices = np.random.choice(data.shape[0], size=num_samples, replace=False)
#             data_sampled = data[random_indices]

            # Create a partially applied function with fixed arguments
            partial_sdtw_div = functools.partial(sdtw_div, gamma=0.1)

            savefilename = 'scg_results/soft-dtw/bootstrap_samples/' + f"bootstrap_nn_{params['n_neighbors']}_md_{params['min_dist']}_win_{window_size}_gamma_01.csv"
#             savefilename = 'scg_results/euclidean/bootstrap_samples/' + f"bootstrap_nn_{params['n_neighbors']}_md_{params['min_dist']}_win_{window_size}_manhattan.csv"

            print(f"Generating bootstrap from the file...")
            start_time = time.time()
            # Apply boot.generate() using the loaded data and the hyperparameters
            out = boot.generate(data, method="umap", B=4, save=savefilename,
                                random_seed=452, random_state=452,
                                num_jobs=4,
                                n_neighbors=params['n_neighbors'],
                                min_dist=params['min_dist'],
                                metric=partial_sdtw_div, low_memory=True
                               )

            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Bootstrap generation time: {elapsed_time} seconds")

            print(f"Calculating variance score...")
            start_time = time.time()

            # Calculate variance score
            variance_score = score.variance(out, method="random", k=50)
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Total variance score calculation time: {elapsed_time} seconds")

            # Save variance scores as a CSV file
            variance_filename = 'scg_results/soft-dtw/var_scores/' + f"var_scores_nn_{params['n_neighbors']}_md_{params['min_dist']}_win_{window_size}_gamma_01.csv"
#             variance_filename = 'scg_results/euclidean/var_scores/' + f"var_scores_nn_{params['n_neighbors']}_md_{params['min_dist']}_win_{window_size}_manhattan.csv"
            print(f"Saving variance scores to file: {variance_filename}")
            variance_score.to_csv(variance_filename, index=False)

            mean_variance_score = variance_score.mean()
            print(f"Mean variance score: {mean_variance_score}")

            # Create a scatter plot using 'x1' and 'x2' columns
            plt.scatter(out['x1'], out['x2'], s=0.2, alpha=0.2, c='grey')
            plt.xlabel('x1')
            plt.ylabel('x2')
            plt.title(f'UMAP projection of window_size: {window_size}, NN: {params["n_neighbors"]}, MD: {params["min_dist"]}, Gamma: 0.1')
            plt.savefig(f'scg_results/soft-dtw/plot_results/umap_plot_nn_{params["n_neighbors"]}_md_{params["min_dist"]}_sdtwGamma01_win_{window_size}.png', dpi=300)
            plt.close()

            # Update the best variance mean and corresponding hyperparameters if a lower variance score is found
            if mean_variance_score < best_variance_mean:
                best_variance_mean = mean_variance_score
                best_params = params

            top_5_scores.append((params, window_size, mean_variance_score))

# Sort the top_5_scores based on variance mean scores in ascending order
top_5_scores = sorted(top_5_scores, key=lambda x: x[2])[:5]

# Print the top 5 hyperparameters and variance mean scores
print("Top 5 Hyperparameters based on Variance Score:")
for params, window_size, variance_mean_score in top_5_scores:
    print("Hyperparameters:", params)
    print("Window Size:", window_size)
    print("Variance Mean Score:", variance_mean_score)
    print()

# Save the top 5 hyperparameters and variance mean scores to a CSV file
csv_filename = 'scg_results/top_hyperparameters.csv'
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Hyperparameters', 'Window Size', 'Variance Mean Score'])
    for params, window_size, variance_mean_score in top_5_scores:
        writer.writerow([params, window_size, variance_mean_score])

In [ ]:
!pip install tslearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.2/358.2 kB 7.2 MB/s eta 0:00:00


In [ ]:
from tslearn.generators import random_walk_blobs
from tslearn.shapelets import LearningShapelets


In [ ]:
X, y = random_walk_blobs(n_ts_per_blob=100, sz=256, d=4, n_blobs=3)
X.shape

(300, 256, 4)

In [ ]:
clf = LearningShapelets(random_state=42).fit_transform(X, y)

/usr/local/lib/python3.10/dist-packages/tslearn/shapelets/shapelets.py:354: FutureWarning: The default value for 'scale' is set to False in version 0.4 to ensure backward compatibility, but is likely to change in a future version.
  warnings.warn("The default value for 'scale' is set to False "


In [ ]:
clf.fit_transform(X)

TypeError: ignored

In [ ]:
partial_sdtw_div = functools.partial(sdtw_div, gamma=0.1)
fit = umap.UMAP(metric=partial_sdtw_div)
%time u = fit.fit_transform(data[:,:, -1])
plt.scatter(u[:,0], u[:,1], c=label, s=0.5, alpha=0.5)
plt.title('UMAP embedding of random colours');